In [1]:
from glob import glob
import os
from scipy.io import loadmat
from scipy import signal,stats
import pandas as pd
import numpy as np
from numpy.fft import fft , ifft
import pywt
import matplotlib.pyplot as plt 
import re

#import mne

#from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs,corrmap)

In [2]:
map_label = [
        {'shoroT1':0,'balaT1':1,'paeinT1':2,'raastT1':3,'chapT1':4,'jeloT1':5,'aqabT1':6,'payanT1':7},
        {'shoroT2':0,'balaT2':1,'paeinT2':2,'raastT2':3,'chapT2':4,'jeloT2':5,'aqabT2':6,'payanT2':7},
        {'shoroT3':0,'balaT3':1,'paeinT3':2,'raastT3':3,'chapT3':4,'jeloT3':5,'aqabT3':6,'payanT3':7},          
]

def filter_noisy_data(data,channel_num,sample_for_data,fs,f_nyq,f_low,f_high):
    
    '''
        در این تابع تمام کارهای فیلتر کردن سیگنال انجام می‌شود.
          به منظور پیاده سازی روند فیلترینگ سیگنال، از فیلتر باترورث پیوسته استفاده شده است که معمولا از نوع میان گذر آن استفاده شده است. 
    '''
    
    data_filter = np.zeros_like(data)
    Wn = [f_low,f_high]/np.float_(f_nyq)
    order_filter = 4
    (b,a) = signal.butter(order_filter, Wn, btype='bandpass', output='ba')

    for channel_signal in range(0,channel_num):
        
        '''
         در این قسمت چون سیگنال ما از ۲۱ کانال گرفته شده است هر کدام از کانال ها را فیلتر می‌کنیم.
         سپس یک فور میزنیم به طول تعداد کانال ها.
         بعد دیتای فیلتر شده را درون یک ماتریس صفر که از قبل تعریف کرده ایم می‌ریزیم.
        '''
        
        data_each_channel = data[channel_signal,:]
        data_filter[channel_signal,:] = signal.filtfilt(b,a,data_each_channel)

    return data_filter





#برای استخراج ویژگی آماری از این تابع استفاده می‌کنیم.
def statistical_features(data_for_feature_extraction):
    
    MEAN = np.mean(data_for_feature_extraction)
    VAR = np.var(data_for_feature_extraction)
    POWER = np.mean(np.power(data_for_feature_extraction,2))
    SKEW = stats.skew(data_for_feature_extraction)
    KUR = stats.kurtosis(data_for_feature_extraction) 
    
    return (MEAN,VAR,POWER,SKEW,KUR)






def time_feature_selection(data,channel_num):
    '''
        داخل آرگومان اول تابع دیتای هر ۵ باند وجود دارد که میتوانیم یکی یکی آن را دریافت کنیم بعد فیچرهای هر کانال هر باند را درون یک ماتریس 
        ذخیره کنیم که در نهایت یک تنسور ۵*۲۱*۵ داریم که ۵ اول تعداد باندهاس،۲۱ تعداد کانال، ۵ تعداد فیچرهای استخراجی می‌باشد.
         ماتریس اول داخل تنسور فیچرهای اسخراج شده از باند دلتا برای ۲۱ کانال است که یک ماتریس ۲۱* ۵ میشه که ۵ تعداد فیچرهای استخراجی می‌باشد و غیره       
    '''
    
    num_feature = 5 #mean var power skew kur 
    time_feature_selections_total_bands = np.zeros((channel_num,num_feature))


        
    for i in range(0,channel_num):
        data_for_feature_extraction = data[i,:]
    
        time_feature_selections_total_bands[i,:] = statistical_features(data_for_feature_extraction)
        

    return time_feature_selections_total_bands
    


def seperate_band(data,channel_num,sample_for_data,fs,f_nyq):
    #در حوزه زمان ریتم های سیگنال را استخراج می‌کنیم
    #یک ماتریس مثلا ۲۱*۳۹۰۰ که همش سیگنال های باند دلتا را دارد
    #برای هر کانال باندهای آن را استخراج می‌کنیم.
    delta = filter_noisy_data(data,channel_num,sample_for_data,fs,f_nyq,f_low=1,f_high=4)
    
    #یک ماتریس مثلا ۲۱*۳۹۰۰ که همش سیگنال های باند تتا را دارد
    theta = filter_noisy_data(data,channel_num,sample_for_data,fs,f_nyq,f_low=4,f_high=8)
    
    #یک ماتریس مثلا ۲۱*۳۹۰۰ که همش سیگنال های باند آلفا را دارد     
    alpha = filter_noisy_data(data,channel_num,sample_for_data,fs,f_nyq,f_low=8,f_high=12)

    #یک ماتریس مثلا ۲۱*۳۹۰۰ که همش سیگنال های باند بتا را دارد
    beta = filter_noisy_data(data,channel_num,sample_for_data,fs,f_nyq,f_low=12,f_high=30)

    #یک ماتریس مثلا ۲۱*۳۹۰۰ که سیگنال های باند گاما در هر کانال را دارد
    gamma = filter_noisy_data(data,channel_num,sample_for_data,fs,f_nyq,f_low=30,f_high=40)

    return (delta,theta,alpha,beta,gamma)





#از این تابع برای تبدیل فوریه گرفتن از سیگنال استفاده می‌کنیم.
def frequency_domain(data):
    #در این قسمت می‌خوایم از سیگنال تبدیل فوریه بگیریم و برای هر کانال این کار را می‌کنیم..
    N = data.shape[1]
    channel_num = data.shape[0]
    matrix_for_save_fft_each_channel = np.zeros_like(data)
    
    for i in range(0,channel_num):
        #به ازای هر کانال یک سیگنال یک اف اف تی می‌گیریم.
        fft_data = fft(data[i,:],N)
        matrix_for_save_fft_each_channel[i,:] = fft_data
    #خروجی: تبدیل فوریه هر ۲۱ کانال در یک ماتریس به طول سیگنال ورودی 
    return matrix_for_save_fft_each_channel


def feature_selection_frequency(data):
    
    N = data.shape[1]
    channel_num = data.shape[0]
    fs = 2000
    f_r = np.linspace(0,fs/2,int(np.floor(N/2)))
    f_r_total = np.concatenate((f_r,f_r[::-1]))

    num_feature = 5 #mean var power skew kur 
    
    feature_selections_total = np.zeros((channel_num,num_feature)) #number_f_bands = 5
    
    for ch_num in range(channel_num):
        #اطلاعات باند فرکانسی یک کانال

        bands_range_frequency = np.abs(data[ch_num,:int(np.floor(N/2))])

        feature_selections_total[ch_num,:] = statistical_features(bands_range_frequency)

    return feature_selections_total

def feature_selectin_wavelet(data,channel_num):
    original_sfreq = 2000
    target_sfreq = 120
    resampling_factor = original_sfreq/target_sfreq
    wavelet = 'db6'
    level = 4
    
    number_of_bands = 5
    number_featuers = 5
    wavelete_feature_extractions = np.zeros((number_of_bands, channel_num, number_featuers))
    for i in range(channel_num):
        
        resampled_eeg_data = signal.resample(data[i,:],int(len(data[i,:])/resampling_factor))
        
        #(a4,d4,d3,d2,d1)
        #(delta,theta,alpha,beta,gamma)
        bands = pywt.wavedec(resampled_eeg_data, wavelet, mode='symmetric', level=level)
        print(len(bands))
    #     for index,band in enumerate(bands):
    #         wavelete_feature_extractions[index,i,:] = statistical_features(band)
    # return wavelete_feature_extractions
        
def read_raw_data(path):
    
    fs = 2000
    f_nyq = fs/2
    
    Temp = loadmat(path)
    a = 7e-7
    data = Temp['EEG_Data']
    
    #determine the label for data
    re_for_get_label=re.findall(r'ID.*event_id',str(Temp['Labels'][0]))[0].split(",")[0]
    re_for_get_label=re_for_get_label.split(':')[1].strip()
    label_for_subject_t = re_for_get_label.replace("'",'')

    #determine channel number eeg signal i
    channel_num = data.shape[0]
    #determine sample number for eeg signal i
    sample_for_data = data.shape[1]
    
    ####################################################denoising data 
    #determine 8 - 30 hz

    data_denoising = filter_noisy_data(data, channel_num, sample_for_data, fs, f_nyq, f_low=8,f_high=30)

    #از این به بعد این دیتای دینویز را برای اسخراج ویژگی به توابع می‌فرستیم
    #####################################################seperate frequency band
    #(delta_time,theta_time,alpha_time,beta_time,gamma_time) = seperate_band(data_denoising,channel_num,sample_for_data,fs,f_nyq)
    
    #####################################################Time Feature Extraction
    time_feature_selection_for_entire_band = time_feature_selection(data_denoising,channel_num)
    
    ############################################################frequency domain + frequency Feature Extraction
    #تبدیل فوریه داده دینویز شده
    change_time_domain_to_frequency_domain_data = frequency_domain(data_denoising)
    #استخراج ویژگی از حوزه فرکانس
    frequency_feature_selection_for_entire_bands = feature_selection_frequency(change_time_domain_to_frequency_domain_data)
    
    #استخراج ویژگی از حوزه ویولت
    #wavelet_feature_selection_for_entire_bands = feature_selectin_wavelet(data_denoising,channel_num)
    
    return label_for_subject_t, time_feature_selection_for_entire_band,frequency_feature_selection_for_entire_bands

In [69]:
path_os = os.getcwd()
PATH = os.path.join(path_os,'data','v1')
subject_T = glob(PATH+f'/s{1}/Directions_and_Time_T{1}/*.mat')
Path_for_save = PATH+f'/s{1}/Directions_and_Time_T{1}/FetureExtraction/'

num_fetures = 5
total_features = 25
chan_numbers = 21 #channel numbers
bands_numbers = 5

data_for_network1 = np.zeros((len(subject_T),chan_numbers*num_fetures))
data_for_network2 = np.zeros((len(subject_T),chan_numbers*num_fetures))
label_y = []

for index,data_raw_path in enumerate(subject_T):
      

    label,TFE,FFE = read_raw_data(data_raw_path)

    '''
        فیچرهای که استخراج کردیم به صورت یک تنسور می‌باشد که هر ماتریس آن اطلاعات یک باند فرکانسی است مثلا ماتریس اول شامل ۲۱ سطر است
        و ۵ ستون که به معنی اطلاعات اسخراج شده ۲۱ کانال است
        الان میخوایم اطلاعات ۵ باند را برای هر کانال پشت سر هم قرار بدهیم..
        
        TFE = Time Feature Extraction
        FFE = Frequency Feature Extraction
        WFE = wavelet Feature Extraction

    '''
    
    if ("T1" in label):
        label_y.append(map_label[0].get(label))
    data_for_network1[index,:] = np.ravel(TFE)
    data_for_network2[index,:] = np.ravel(FFE)
    concat = np.concatenate((data_for_network1,data_for_network2),axis=1)
    # concatenate_TFE_for_each_channel = np.zeros((chan_numbers,total_features))
    # concatenate_FFE_for_each_channel = np.zeros((chan_numbers,total_features))
    # concatenate_WFE_for_each_channel = np.zeros((chan_numbers,total_features))

    
    # for chan_number in range(chan_numbers):
    
    #     LB = 0
    #     HB = 5
        
    #     for band in range(bands_numbers):
    #         concatenate_TFE_for_each_channel[chan_number,LB:HB] =   TFE[band,chan_number,:]
    #         concatenate_FFE_for_each_channel[chan_number,LB:HB] =   FFE[band,chan_number,:]
    #         concatenate_WFE_for_each_channel[chan_number,LB:HB] =   WFE[band,chan_number,:]
    #         LB += 5
    #         HB += 5

    # name = data_raw_path.split('/')[-1].split('.')[0]
    # np.save(Path_for_save+"/TFE/"+f"{index}_"+name+"-TFE-"+label,concatenate_TFE_for_each_channel)
    # np.save(Path_for_save+"/FFE/"+f"{index}_"+name+"-FFE-"+label,concatenate_FFE_for_each_channel)
    # np.save(Path_for_save+"/WFE/"+f"{index}_"+name+"-WFE-"+label,concatenate_WFE_for_each_channel)

/tmp/ipykernel_26500/630292713.py:108: ComplexWarning: Casting complex values to real discards the imaginary part
  matrix_for_save_fft_each_channel[i,:] = fft_data


In [6]:
y = np.array(label_y)

In [7]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
from sklearn.feature_selection import RFE
import tensorflow as tf
from tensorflow import keras 
from keras.utils import to_categorical

In [80]:
X_train,X_test,y_train,y_test = train_test_split(concat,y,test_size=0.2,random_state=42)

MEAN_Train = X_train.mean(axis=1)
STD_Train = np.std(X_train,axis=1)

Mean_train = np.zeros(X_train.shape)
Std_train = np.zeros(X_train.shape)


for i in range(X_train.shape[0]):
    Mean_train[i,:] = np.repeat(MEAN_Train[i],X_train.shape[1])
    Std_train[i,:] = np.repeat(STD_Train[i],X_train.shape[1])
X_train = (X_train - Mean_train)/Std_train

MEAN_Test = X_test.mean(axis=1)
STD_Test = np.std(X_test,axis=1)

Mean_test = np.zeros(X_test.shape)
Std_test = np.zeros(X_test.shape)

for i in range(X_test.shape[0]):
    Mean_test[i,:] = np.repeat(MEAN_Test[i],X_test.shape[1])
    Std_test[i,:] = np.repeat(STD_Test[i],X_test.shape[1])
X_test = (X_test - Mean_test)/Std_test

In [72]:
y = np.array(label_y)

In [81]:
y = to_categorical(y,num_classes=8)
model = keras.Sequential()
model.add(keras.layers.Dense(64,input_dim=X_train.shape[1],activation='relu'))
model.add(keras.layers.Dropout(.5))
model.add(keras.layers.Dense(32,activation='relu'))
model.add(keras.layers.Dense(8,activation='softmax'))
# model.compile(loss='categorical_crossentropy',optimizer='adam',matrics=['accuracy'])
model.compile(optimizer="Adam", loss="mse", metrics=["mae", "acc"])

In [82]:
model.fit(X_train,y_train,epochs=1000,batch_size=32)

Epoch 1/1000
2/2 [==============================] - 0s 2ms/step - loss: 0.1274 - mae: 0.2207 - acc: 0.0938
Epoch 2/1000
2/2 [==============================] - 0s 2ms/step - loss: 0.1192 - mae: 0.2181 - acc: 0.1250
Epoch 3/1000
2/2 [==============================] - 0s 2ms/step - loss: 0.1128 - mae: 0.2159 - acc: 0.1875
Epoch 4/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.1177 - mae: 0.2218 - acc: 0.0781
Epoch 5/1000
2/2 [==============================] - 0s 2ms/step - loss: 0.1137 - mae: 0.2183 - acc: 0.1094
Epoch 6/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.1112 - mae: 0.2165 - acc: 0.1875
Epoch 7/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.1111 - mae: 0.2168 - acc: 0.1562
Epoch 8/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.1115 - mae: 0.2180 - acc: 0.1562
Epoch 9/1000
2/2 [==============================] - 0s 9ms/step - loss: 0.1116 - mae: 0.2180 - acc: 0.1406
Epoch 10/1000
2/2 [==================

In [83]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred,axis=1)


1/1 [==============================] - 0s 36ms/step


In [84]:
accuracy_ = accuracy_score(np.argmax(y_test,axis=1),y_pred_classes)
class_report = classification_report(np.argmax(y_test,axis=1),y_pred_classes)

/home/nika/Documents/EEG/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/nika/Documents/EEG/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/nika/Documents/EEG/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [85]:
print(accuracy_)

0.1875


In [86]:
np.argmax(y_test,axis=1)

array([0, 1, 1, 6, 2, 0, 4, 0, 1, 7, 5, 0, 3, 0, 0, 2])

In [87]:
y_pred_classes

array([7, 5, 3, 3, 5, 3, 3, 0, 3, 7, 2, 3, 2, 0, 7, 3])

In [36]:
a = np.array([[1,2,3],[4,5,6]])

In [40]:
np.mean(a,axis=1)

array([2., 5.])

In [42]:
np.repeat(np.mean(a,axis=1),(3,3))

array([2., 2., 2., 5., 5., 5.])

In [58]:
a=X_train.mean(axis=1)
b = np.std(X_train,axis=1)

In [51]:
a.shape

(56,)

In [59]:
qq = np.zeros(X_train.shape)
ww = np.zeros(X_train.shape)
for i in range(X_train.shape[0]):
    qq[i,:] = np.repeat(a[i],105)
    ww[i,:] = np.repeat(a[i],105)
(X_train-qq)/ww

array([[-0.99929129,  8.27674699,  8.33689477, ...,  1.10899764,
        -0.99980309, -0.99382665],
       [-0.9987019 ,  1.49105236,  1.53765977, ...,  1.255472  ,
        -0.99963273, -0.9957015 ],
       [-0.99916677,  2.03129287,  2.08433621, ...,  1.02950258,
        -0.99988112, -0.99882932],
       ...,
       [-0.99945479,  7.48588674,  7.53066299, ...,  0.43288804,
        -0.99989263, -0.99783598],
       [-0.99909878,  5.06557197,  5.14157304, ...,  1.87111947,
        -0.99980689, -0.99596386],
       [-0.99913917,  1.60422053,  1.65983997, ...,  1.02325305,
        -0.9998666 , -0.99848109]])

In [61]:
np.array([[6,2,4,4]])/np.array([[2,2,2,2]])

array([[3., 1., 2., 2.]])

In [55]:
qq.shape

(56, 105)

In [56]:
a.shape

(56,)

In [79]:
Std_train

array([[ 59792.24733194,  59792.24733194,  59792.24733194, ...,
         59792.24733194,  59792.24733194,  59792.24733194],
       [ 74671.48850405,  74671.48850405,  74671.48850405, ...,
         74671.48850405,  74671.48850405,  74671.48850405],
       [ 72993.32692808,  72993.32692808,  72993.32692808, ...,
         72993.32692808,  72993.32692808,  72993.32692808],
       ...,
       [207840.18074466, 207840.18074466, 207840.18074466, ...,
        207840.18074466, 207840.18074466, 207840.18074466],
       [104997.18131376, 104997.18131376, 104997.18131376, ...,
        104997.18131376, 104997.18131376, 104997.18131376],
       [ 76840.45257207,  76840.45257207,  76840.45257207, ...,
         76840.45257207,  76840.45257207,  76840.45257207]])

In [89]:
np.std(X_train)

1.0